In [1]:
import xarray as xr

In [3]:
import dask
import distributed

with dask.config.set({"distributed.scheduler.worker-saturation": 1.0,
                      "distributed.nanny.pre-spawn-environ.MALLOC_TRIM_THRESHOLD_": 0}):
    client = distributed.Client()

In [4]:
ds = xr.tutorial.open_dataset('air_temperature')

In [5]:
ds

<xarray.Dataset> Size: 15MB
Dimensions:  (lat: 25, time: 2920, lon: 53)
Coordinates:
  * lat      (lat) float32 100B 75.0 72.5 70.0 67.5 65.0 ... 22.5 20.0 17.5 15.0
  * lon      (lon) float32 212B 200.0 202.5 205.0 207.5 ... 325.0 327.5 330.0
  * time     (time) datetime64[ns] 23kB 2013-01-01 ... 2014-12-31T18:00:00
Data variables:
    air      (time, lat, lon) float32 15MB ...
Attributes:
    Conventions:  COARDS
    title:        4x daily NMC reanalysis (1948)
    description:  Data is from NMC initialized reanalysis\n(4x/day).  These a...
    platform:     Model
    references:   http://www.esrl.noaa.gov/psd/data/gridded/data.ncep.reanaly...

In [15]:
ds.groupby('time.month').mean(dim='time',engine='flox',method='cohorts')

<xarray.Dataset> Size: 64kB
Dimensions:  (month: 12, lat: 25, lon: 53)
Coordinates:
  * lat      (lat) float32 100B 75.0 72.5 70.0 67.5 65.0 ... 22.5 20.0 17.5 15.0
  * lon      (lon) float32 212B 200.0 202.5 205.0 207.5 ... 325.0 327.5 330.0
  * month    (month) int64 96B 1 2 3 4 5 6 7 8 9 10 11 12
Data variables:
    air      (month, lat, lon) float32 64kB 246.3 246.4 246.2 ... 297.6 297.5
Attributes:
    Conventions:  COARDS
    title:        4x daily NMC reanalysis (1948)
    description:  Data is from NMC initialized reanalysis\n(4x/day).  These a...
    platform:     Model
    references:   http://www.esrl.noaa.gov/psd/data/gridded/data.ncep.reanaly...

In [48]:
var_string = list(ds.data_vars.keys())[0]
result_DS = xr.merge([ds.groupby('time.month').mean(dim='time',engine='flox',method='cohorts').rename({var_string:'mean_'+var_string}),
                      ds.groupby('time.month').min(dim='time',engine='flox',method='cohorts').rename({var_string:'min_'+var_string}),
                      ds.groupby('time.month').max(dim='time',engine='flox',method='cohorts').rename({var_string:'max_'+var_string}),
                      ds.groupby('time.month').std(dim='time',engine='flox',method='cohorts').rename({var_string:'std_'+var_string}),
                      ds.groupby('time.month').median(dim='time').rename({var_string:'median_'+var_string})
])
result_DS

<xarray.Dataset> Size: 318kB
Dimensions:     (month: 12, lat: 25, lon: 53)
Coordinates:
  * lat         (lat) float32 100B 75.0 72.5 70.0 67.5 ... 22.5 20.0 17.5 15.0
  * lon         (lon) float32 212B 200.0 202.5 205.0 207.5 ... 325.0 327.5 330.0
  * month       (month) int64 96B 1 2 3 4 5 6 7 8 9 10 11 12
Data variables:
    mean_air    (month, lat, lon) float32 64kB 246.3 246.4 246.2 ... 297.6 297.5
    min_air     (month, lat, lon) float32 64kB 227.6 228.6 228.6 ... 295.0 294.4
    max_air     (month, lat, lon) float32 64kB 260.7 260.7 260.6 ... 299.7 299.7
    std_air     (month, lat, lon) float32 64kB 5.848 5.61 5.414 ... 0.9791 1.071
    median_air  (month, lat, lon) float32 64kB 246.0 246.4 246.3 ... 297.7 297.7
Attributes:
    Conventions:  COARDS
    title:        4x daily NMC reanalysis (1948)
    description:  Data is from NMC initialized reanalysis\n(4x/day).  These a...
    platform:     Model
    references:   http://www.esrl.noaa.gov/psd/data/gridded/data.ncep.reanaly...

In [49]:
var_string = list(ds.data_vars.keys())[0]
quant = ds.groupby('time.month').quantile([0.05,0.95],skipna=False,dim='time')
quant

<xarray.Dataset> Size: 255kB
Dimensions:   (lat: 25, month: 12, quantile: 2, lon: 53)
Coordinates:
  * lat       (lat) float32 100B 75.0 72.5 70.0 67.5 ... 22.5 20.0 17.5 15.0
  * lon       (lon) float32 212B 200.0 202.5 205.0 207.5 ... 325.0 327.5 330.0
  * quantile  (quantile) float64 16B 0.05 0.95
  * month     (month) int64 96B 1 2 3 4 5 6 7 8 9 10 11 12
Data variables:
    air       (month, quantile, lat, lon) float64 254kB 237.5 237.8 ... 299.0

In [50]:
quant_ds = xr.merge([quant.isel(quantile=0).reset_coords(drop=True).rename({var_string:'quantile_05_'+var_string}),quant.isel(quantile=1).reset_coords(drop=True).rename({var_string:'quantile_95_'+var_string})])
quant_ds

<xarray.Dataset> Size: 255kB
Dimensions:          (lat: 25, month: 12, lon: 53)
Coordinates:
  * lat              (lat) float32 100B 75.0 72.5 70.0 67.5 ... 20.0 17.5 15.0
  * lon              (lon) float32 212B 200.0 202.5 205.0 ... 325.0 327.5 330.0
  * month            (month) int64 96B 1 2 3 4 5 6 7 8 9 10 11 12
Data variables:
    quantile_05_air  (month, lat, lon) float64 127kB 237.5 237.8 ... 295.9 295.7
    quantile_95_air  (month, lat, lon) float64 127kB 257.1 255.8 ... 299.0 299.0

In [51]:
result_DS = xr.merge([result_DS,quant_ds])
result_DS

<xarray.Dataset> Size: 573kB
Dimensions:          (month: 12, lat: 25, lon: 53)
Coordinates:
  * lat              (lat) float32 100B 75.0 72.5 70.0 67.5 ... 20.0 17.5 15.0
  * lon              (lon) float32 212B 200.0 202.5 205.0 ... 325.0 327.5 330.0
  * month            (month) int64 96B 1 2 3 4 5 6 7 8 9 10 11 12
Data variables:
    mean_air         (month, lat, lon) float32 64kB 246.3 246.4 ... 297.6 297.5
    min_air          (month, lat, lon) float32 64kB 227.6 228.6 ... 295.0 294.4
    max_air          (month, lat, lon) float32 64kB 260.7 260.7 ... 299.7 299.7
    std_air          (month, lat, lon) float32 64kB 5.848 5.61 ... 0.9791 1.071
    median_air       (month, lat, lon) float32 64kB 246.0 246.4 ... 297.7 297.7
    quantile_05_air  (month, lat, lon) float64 127kB 237.5 237.8 ... 295.9 295.7
    quantile_95_air  (month, lat, lon) float64 127kB 257.1 255.8 ... 299.0 299.0
Attributes:
    Conventions:  COARDS
    title:        4x daily NMC reanalysis (1948)
    description:  Data is from NMC initialized reanalysis\n(4x/day).  These a...
    platform:     Model
    references:   http://www.esrl.noaa.gov/psd/data/gridded/data.ncep.reanaly...